#### Definition of the model

* finding the optimal parameters
* finding the optimal equations for mu and qs
* applying **Latin Hypercube Sampling** for generating parameter sets
* calculating the **RMSE** of model and experiment

#### Code

In [1]:
import pandas as pd
import numpy as np
import yaml
from sklearn.metrics import mean_squared_error
from C_functions_opti import get_LHS_samples, model_optimization, model_optimization2, plot_estimation

In [2]:
# Load experimental data
df_exp = pd.read_csv('data/data_combined.csv')
biomass_exp = df_exp['Biomass [g/L]']
substrate_exp = df_exp['Glucose [g/L]']

In [3]:
# Load parameters from YAML file
with open('config/parameters.yml', 'r') as file:
    param = yaml.safe_load(file)

The growth rate and the substrate uptake rate are dependend of each other - meaning that if we define a equation for **mu**, then **qs** is defined by mu with $qs=mu/Yxs$. Contrarily, when **qs** is defined by an ODE, then **mu** is calculated with $mu = qs * Yxs$.

In [4]:
mu0 = lambda mu_max, c_glucose, Ks: mu_max * c_glucose / (c_glucose + Ks) # -- MONOD
mu1 = lambda mu_max, c_biomass, X_max: mu_max * (1 - (c_biomass/ X_max)) # -- LOGISTIC
mu2 = lambda mu_max, c_glucose, Ks, c_biomass, X_max: mu_max * (c_glucose / (c_glucose + Ks)) * (1 - (c_biomass/ X_max)) # -- MONOD + LOGISTIC
mu3 = lambda mu_max, c_glucose, Ks, Ki, c_biomass, X_max: mu_max * (c_glucose / (c_glucose + Ks + (c_glucose**2/ Ki))) * (1 - (c_biomass/ X_max)) # -- MONOD + LOGISTIC + INHIBITION
mu4 = lambda qs, Yxs: qs * Yxs # for changing qs

In [5]:
qs0 = lambda qs_max, c_glucose, Ks_qs: qs_max * c_glucose / (Ks_qs + c_glucose) # -- MONOD
qs1 = lambda qs_max, c_glucose, Ks_qs, Ki, glu_met: qs_max * c_glucose / (Ks_qs + c_glucose) * (Ki / (Ki + glu_met)) # -- MONOD + NON COMPETITIVE INHIBITION
qs2 = lambda qs_max, c_glucose, Ks_qs, c_biomass, lag: qs_max * c_glucose / (Ks_qs + c_glucose) * (1 / (np.exp(c_biomass * lag))) # -- MONOD + METABOLIZED GLU
qs3 = lambda mu, Yxs: mu / Yxs # for changing mu

In [6]:
# Saving all equations in each list
mu_all=[mu0, mu1, mu2, mu3, mu4]
qs_all=[qs0, qs1, qs2, qs3]

In [7]:
# Root mean squared error is the objective function
def objective_function(parameters, mu_eq, num_mu, qs_eq, num_qs):
    # Solve the model using the optimal parameters
    time_pred, biomass_pred, substrate_pred, volume_pred = model_optimization(param, parameters, mu_eq, num_mu, qs_eq, num_qs)  # Solve the model using the current parameters
    biomass = pd.concat([biomass_exp, pd.Series(biomass_pred)], axis=1, keys=['biomass_exp', 'biomass_pred']).dropna()
    biomass_exp_ = biomass['biomass_exp'].values
    biomass_pred_ = biomass['biomass_pred'].values
    mse_x = mean_squared_error(biomass_exp_, biomass_pred_)  # Calculate mean squared error for biomass

    glucose = pd.concat([substrate_exp, pd.Series(substrate_pred)], axis=1, keys=['substrate_exp', 'substrate_pred']).dropna()
    substrate_exp_ = glucose['substrate_exp'].values
    substrate_pred_ = glucose['substrate_pred'].values
    mse_s = mean_squared_error(substrate_exp_, substrate_pred_)  # Calculate mean squared error for substrate
    
    # Calculate the combined rmse
    mse = (mse_x + mse_s)/2
    rmse = np.sqrt(mse)  # Calculate root mean squared error
    return rmse, time_pred, biomass_pred, substrate_pred, volume_pred

In [8]:
# Root mean squared error is the objective function
def objective_function2(parameters, mu_eq, num_mu, qs_eq, num_qs):
    # Solve the model using the optimal parameters
    time_pred, biomass_pred, substrate_pred, volume_pred = model_optimization2(param, parameters, mu_eq, num_mu, qs_eq, num_qs)  # Solve the model using the current parameters
    biomass = pd.concat([biomass_exp, pd.Series(biomass_pred)], axis=1, keys=['biomass_exp', 'biomass_pred']).dropna()
    biomass_exp_ = biomass['biomass_exp'].values
    biomass_pred_ = biomass['biomass_pred'].values
    mse_x = mean_squared_error(biomass_exp_, biomass_pred_)  # Calculate mean squared error for biomass

    glucose = pd.concat([substrate_exp, pd.Series(substrate_pred)], axis=1, keys=['substrate_exp', 'substrate_pred']).dropna()
    substrate_exp_ = glucose['substrate_exp'].values
    substrate_pred_ = glucose['substrate_pred'].values
    mse_s = mean_squared_error(substrate_exp_, substrate_pred_)  # Calculate mean squared error for substrate
    
    # Calculate the combined rmse
    mse = (mse_x + mse_s)/2
    rmse = np.sqrt(mse)  # Calculate root mean squared error
    return rmse, time_pred, biomass_pred, substrate_pred, volume_pred

In [9]:
'''
## Version 1: Define all parameters manually in the config/parameters.yml file

# Extract the parameter combination and the number of the set in order to save it in the correct folder
est_mu_max = param['est_mu_max'] # ['set0']

# -------------------------------

## Version 2: Apply LH sampling on all parameters

# Set the number of samples and parameters
num_samples = 50
num_parameters = 9

# Define the ranges for each parameter
parameter_bounds = [
    [0.01, 0.6],    # Range for parameter 1 mu_max
    [18, 25],       # Range for parameter 2 X_max
    [0.1, 10.0],    # Range for parameter 3 - Ks
    [0.1, 10.0],    # Range for parameter 4 - Ks_qs
    [6.0, 8.0],     # Range for parameter 5 - Ki
    [0.2, 0.6],     # Range for parameter 6 - Yxs
    [0.5, 1.5],     # Range for parameter 7 - qs_max
    [0.05, 0.5],   # Range for parameter 8 - m_s
    [0.001, 0.02],  # Range for parameter 9 - lag
]
'''

"\n## Version 1: Define all parameters manually in the config/parameters.yml file\n\n# Extract the parameter combination and the number of the set in order to save it in the correct folder\nest_mu_max = param['est_mu_max'] # ['set0']\n\n# -------------------------------\n\n## Version 2: Apply LH sampling on all parameters\n\n# Set the number of samples and parameters\nnum_samples = 50\nnum_parameters = 9\n\n# Define the ranges for each parameter\nparameter_bounds = [\n    [0.01, 0.6],    # Range for parameter 1 mu_max\n    [18, 25],       # Range for parameter 2 X_max\n    [0.1, 10.0],    # Range for parameter 3 - Ks\n    [0.1, 10.0],    # Range for parameter 4 - Ks_qs\n    [6.0, 8.0],     # Range for parameter 5 - Ki\n    [0.2, 0.6],     # Range for parameter 6 - Yxs\n    [0.5, 1.5],     # Range for parameter 7 - qs_max\n    [0.05, 0.5],   # Range for parameter 8 - m_s\n    [0.001, 0.02],  # Range for parameter 9 - lag\n]\n"

#### First approach

In [10]:
# Set the number of samples and parameters
num_samples = 100
num_parameters = 9

# Define the ranges for each parameter
parameter_bounds = [
    [0.2, 0.4],    # Range for parameter 1 mu_max
    [15, 25],       # Range for parameter 2 X_max
    [0.1, 20.0],    # Range for parameter 3 - Ks
    [0.1, 20.0],    # Range for parameter 4 - Ks_qs
    [0.1, 20.0],     # Range for parameter 5 - Ki
    [0.3, 0.5],     # Range for parameter 6 - Yxs
    [0.1, 1.5],     # Range for parameter 7 - qs_max
    [0.0, 0.2],   # Range for parameter 8 - m_s
    [0.001, 1.0],  # Range for parameter 9 - lag
]

In [11]:
LHS_samples = get_LHS_samples(num_samples, num_parameters, parameter_bounds)
LHS_samples.shape

(100, 9)

In [12]:
df_all_sets = pd.DataFrame(columns=['set', 'mu', 'qs', 'mu_max', 'X_max', 'Ks', 'Ks_qs', 'Ki', 'Yxs', 'qs_max', 'm_s', 'lag', 'rmse'])
for set_num in range(LHS_samples.shape[0]):
    # Save all parameters and equations and the RMSE in a dataframe
    ## with the beginning of one set a new rmse_overview will be created
    rmse_one_set = []
    #key = f'set{set_num}' ; init_p = est_mu_max[key]
    init_p = list(LHS_samples[set_num, :])
    for i in range(len(mu_all)):
        # Define the combination of equations
        mu_eq = mu_all[i]; num_mu = i

        # mu0 to mu3 are combined with qs3 while for mu4 all qs except for qs3 are combined
        if i != 4:
            j = 3
            qs_eq = qs_all[j]; num_qs = j   

            # Make the predictions and calculate the error
            rmse, time_pred, biomass_pred, substrate_pred, volume_pred = objective_function(init_p, mu_eq, num_mu, qs_eq, num_qs)
            # save the parameters in a dataframe
            append_list=[set_num, i, j, init_p[0], init_p[1], init_p[2], init_p[3], init_p[4], init_p[5], init_p[6], init_p[7], init_p[8], round(rmse, 3)]
            rmse_one_set.append(append_list)

            # Make a plot and save it
            title = f'set{set_num}/ mu{i} - qs{j} - rmse: {round(rmse, 3)}'
            plot_name = f'set{set_num}_mu{i}_qs{j}_rmse{int(rmse)}'

            if rmse <= float(6):
                plot_estimation(time_pred, biomass_pred, substrate_pred, volume_pred, title, plot_name, set_num)
        
        elif i == 4:
            for j in range(3):
                qs_eq = qs_all[j]; num_qs = j   

                # Make the predictions and calculate the error
                rmse, time_pred, biomass_pred, substrate_pred, volume_pred = objective_function(init_p, mu_eq, num_mu, qs_eq, num_qs)
                # save the parameters in a dataframe
                append_list=[set_num, i, j, init_p[0], init_p[1], init_p[2], init_p[3], init_p[4], init_p[5], init_p[6], init_p[7], init_p[8], round(rmse, 3)]
                rmse_one_set.append(append_list)

                # Make a plot and save it
                title = f'set{set_num}/ mu{i} - qs{j} - rmse: {round(rmse, 3)}'
                plot_name = f'set{set_num}_mu{i}_qs{j}_rmse{int(rmse)}'

                if rmse <= float(6):
                    plot_estimation(time_pred, biomass_pred, substrate_pred, volume_pred, title, plot_name, set_num)

    # save the parameters of one set
    df_1set = pd.DataFrame(rmse_one_set, columns=['set', 'mu', 'qs', 'mu_max', 'X_max', 'Ks', 'Ks_qs', 'Ki', 'Yxs', 'qs_max', 'm_s', 'lag', 'rmse'])
    df_all_sets = pd.concat([df_all_sets,df_1set], axis=0, ignore_index=True)


df_all_sets.sort_values(by=['rmse'], ascending=True, inplace=True)
df_all_sets.to_csv(f'data/estimation/new_eq/data.csv')

df_all_sets.head(10)

,set,mu,qs,mu_max,X_max,Ks,Ks_qs,Ki,Yxs,qs_max,m_s,lag,rmse
330,47,1,3,0.223009,15.284251,12.815792,13.666199,8.385124,0.488003,0.185515,0.126018,0.960372,5.152
645,92,1,3,0.204009,15.737607,4.425793,12.329298,18.076694,0.453510,0.161783,0.009940,0.186247,5.274
316,45,1,3,0.203832,16.353882,14.056443,11.743992,7.460560,0.475052,1.407734,0.094469,0.290482,5.279
237,33,4,2,0.215200,21.090769,12.386271,7.454611,7.949798,0.354449,1.357136,0.002214,0.231681,5.433
519,74,1,3,0.285389,15.387907,19.788794,18.205085,17.028435,0.436128,0.542645,0.097103,0.565444,5.487
638,91,1,3,0.229415,16.221708,15.771856,9.523837,7.821250,0.449497,0.707306,0.135393,0.073825,5.522
57,8,1,3,0.235634,17.018607,7.126090,10.705084,3.332277,0.487560,0.301303,0.023576,0.926177,5.532
185,26,3,3,0.363034,20.548446,2.642712,8.947694,19.505286,0.454635,1.157531,0.196764,0.739613,5.553
1,0,1,3,0.319900,15.095500,9.469292,5.354897,10.322082,0.414484,1.115024,0.111527,0.640640,5.560
269,38,3,3,0.376917,18.411964,3.425353,19.396092,18.876010,0.362747,0.241350,0.122569,0.898287,5.591


In [13]:
df_LHS = pd.read_csv('data/estimation/LHS_sampling/data.csv')

#### Second approach

In [14]:
# Set the number of samples and parameters
num_samples = 100
num_parameters = 4

# Define the ranges for each parameter
parameter_bounds = [
    [0.2, 0.5], # mu_max
    [15, 25], # X_max
    [0.3, 0.5], # Yxs
    [0.01, 1.5] # m_s
]

LHS_samples = get_LHS_samples(num_samples, num_parameters, parameter_bounds)
LHS_samples.shape

(100, 4)

In [15]:
df_all_sets = pd.DataFrame(columns=['set', 'mu', 'qs', 'mu_max', 'X_max', 'Yxs', 'm_s', 'rmse'])
for set_num in range(LHS_samples.shape[0]):
    # Save all parameters and equations and the RMSE in a dataframe
    ## with the beginning of one set a new rmse_overview will be created
    rmse_one_set = []
    #key = f'set{set_num}' ; init_p = est_mu_max[key]
    init_p = list(LHS_samples[set_num, :])
    
    i = 1; j = 3
    mu_eq = mu_all[i]; num_mu = i
    qs_eq = qs_all[j]; num_qs = j   

    # Make the predictions and calculate the error
    rmse, time_pred, biomass_pred, substrate_pred, volume_pred = objective_function2(init_p, mu_eq, num_mu, qs_eq, num_qs)
    # save the parameters in a dataframe
    append_list=[set_num, i, j, init_p[0], init_p[1], init_p[2], init_p[3], round(rmse, 3)]
    rmse_one_set.append(append_list)

    # Make a plot and save it
    title = f'set{set_num}/ mu{i} - qs{j} - rmse: {round(rmse, 3)}'
    plot_name = f'set{set_num}_mu{i}_qs{j}_rmse{int(rmse)}'

    if rmse <= float(6):
        plot_estimation(time_pred, biomass_pred, substrate_pred, volume_pred, title, plot_name, set_num)
    
    # save the parameters of one set
    df_1set = pd.DataFrame(rmse_one_set, columns=['set', 'mu', 'qs', 'mu_max', 'X_max', 'Yxs', 'm_s', 'rmse'])
    df_all_sets = pd.concat([df_all_sets,df_1set], axis=0, ignore_index=True)


df_all_sets.sort_values(by=['rmse'], ascending=True, inplace=True)
df_all_sets.to_csv(f'data/estimation/2906_1/data_2906.csv')

df_all_sets.head(10)

,set,mu,qs,mu_max,X_max,Yxs,m_s,rmse
21,21,1,3,0.414248,15.924432,0.476219,0.030740,6.203
41,41,1,3,0.436665,16.520348,0.444248,0.081710,8.696
14,14,1,3,0.368556,18.810708,0.390581,0.050352,9.456
91,91,1,3,0.475495,24.849595,0.367556,0.012703,10.637
92,92,1,3,0.220361,22.804384,0.454928,0.059377,11.457
17,17,1,3,0.403651,16.448260,0.394801,0.125709,12.022
18,18,1,3,0.377979,17.047872,0.432809,0.129507,12.476
4,4,1,3,0.496711,18.397958,0.349177,0.105617,12.872
31,31,1,3,0.359044,21.122703,0.306752,0.091823,16.672
96,96,1,3,0.421654,15.684733,0.479006,0.255291,18.780
